In [0]:
src='/content/drive/My Drive/classification_Dataset/hackerearth_dataset.zip'

dest='/content/drive/My Drive/classification_Dataset/hackerearth_data/'

In [0]:
#!unzip '/content/drive/My Drive/classification_Dataset/hackerearth_dataset.zip' -d '/content/drive/My Drive/classification_Dataset/hackerearth_data/'

In [0]:
# import pandas as pd

# df = pd.read_csv('/content/drive/My Drive/classification_Dataset/cancer_classification.csv')
# df.head()

In [0]:
cd dest


/content/dest


In [0]:
cd '/content/drive/My Drive/classification_Dataset/hackerearth_data/'

/content/drive/My Drive/classification_Dataset/hackerearth_data


In [0]:
ls


dataset/


In [0]:
cd dataset

/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset


In [0]:
ls

 test.csv  'Test Images'/   train.csv  'Train Images'/


In [0]:
cd Train\ Images

/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset/Train Images


In [0]:
pwd

'/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset'

In [0]:
train_images='/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset/Train Images/'
train_csv= '/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset/train.csv'

test_images='/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset/Test Images/'
test_csv= '/content/drive/My Drive/classification_Dataset/hackerearth_data/dataset/test.csv'

model_path='/content/drive/My Drive/classification_Dataset/'
image_dims=[224,224,3]
batch_size=16
epochs=10



In [0]:

import matplotlib.pyplot as plt 
import cv2 
import imutils
from os import listdir
import os
%matplotlib inline

import sys
# sys.executable
import pandas as pd
import keras as K
from  keras.applications.inception_v3 import InceptionV3

In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

# set the matplotlib backend so figures can be saved in the background
import matplotlib
# matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
import pandas as pd

import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [57]:

data=pd.read_csv(train_csv)
data.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [56]:
#one hot encoding of data
dummy_cols=pd.Series(data['Class'].unique())

label_dict=pd.get_dummies(dummy_cols).astype(int)
label_dict

,Attire,Decorationandsignage,Food,misc
0,0,0,1,0
1,0,0,0,1
2,1,0,0,0
3,0,1,0,0


In [34]:
print(dummy_cols)

0                    Food
1                    misc
2                  Attire
3    Decorationandsignage
dtype: object


In [44]:

imgs=[]
for i,file in enumerate(listdir(train_images)):
#     print(file)
    img=cv2.imread(train_images+file)
    print(img.shape)
    imgs.append(img)
    if i ==10:
        break

(108, 80, 3)
(120, 80, 3)
(60, 80, 3)
(60, 80, 3)
(60, 80, 3)
(60, 80, 3)
(111, 80, 3)
(60, 80, 3)
(60, 80, 3)
(113, 80, 3)
(60, 80, 3)


In [0]:
def generate_image(batch_size=16,train=True, path=train_csv, img_path=train_images, label_dict=label_dict):
    image_paths=sorted(list(listdir(train_images)))

    random.seed(10)

    random.shuffle(image_paths)

    train_paths=image_paths[:4500]

    test_paths=image_paths[4500:]

    data=pd.read_csv(path)

    if train==False:
        image_paths=test_paths

    csv_data=dict(zip(data['Image'].values,data['Class'].values))

    index=0
    while True:
#         index*batch_size<len(image_paths):
        batch_x=[]
        batch_y=[]
        batch_imgs=np.random.choice(image_paths,size=batch_size)
#         image_paths[(index)*batch_size:(index+1)*batch_size]
        for i in batch_imgs:
            temp_img=cv2.imread(train_images+i)
            temp_img=cv2.resize(temp_img,(image_dims[0],image_dims[1]))
            batch_x.append(img_to_array(temp_img))
            batch_y.append(label_dict[csv_data[i]])
        x=np.array(batch_x, dtype='float')/255
        y=np.array(batch_y)
        index+=1
        yield(x,y)

In [0]:
def generate_image_test(batch_size=16,train=True, path=train_csv, img_path=train_images, label_dict=label_dict):
    image_paths=sorted(list(listdir(train_images)))

    random.seed(10)

    random.shuffle(image_paths)

    train_paths=image_paths[:4500]

    test_paths=image_paths[4500:]

    data=pd.read_csv(path)

    if train==False:
        image_paths=test_paths

    csv_data=dict(zip(data['Image'].values,data['Class'].values))
    
    index=0
    while True:
#         index*batch_size<len(image_paths):
        batch_x=[]
        batch_y=[]
        batch_imgs=np.random.choice(image_paths,size=batch_size)
#         image_paths[(index)*batch_size:(index+1)*batch_size]
        for i in batch_imgs:
            temp_img=cv2.imread(train_images+i)
            temp_img=cv2.resize(temp_img,(image_dims[0],image_dims[1]))
            batch_x.append(img_to_array(temp_img))
            batch_y.append(label_dict[csv_data[i]])
        x=np.array(batch_x, dtype='float')/255
        y=np.array(batch_y)
        index+=1
        yield(x,y)

In [0]:
vggmodel = VGG16(weights='imagenet', include_top=True)

In [46]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [66]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [67]:
from keras.models import Model
X= vggmodel.layers[-2].output
predictions = Dense(4, activation="softmax")(X)
model_final = Model(input = vggmodel.input, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  after removing the cwd from sys.path.


In [50]:
model_final.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras import optimizers
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [72]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

BS= batch_size

checkpoint = ModelCheckpoint("/content/drive/My Drive/classification_Dataset/hackerearth_tag.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

model_final.fit_generator(generator=generate_image(),
        validation_data=generate_image_test(train=False),
        validation_steps=1000//BS,
        steps_per_epoch=4500// BS,
        epochs=epochs, verbose=1,callbacks=[checkpoint,early])

model_final.save_weights("/content/drive/My Drive/classification_Dataset/hackerearth_tag.h5")

Epoch 1/10
281/281 [==============================] - 34s 122ms/step - loss: 1.1965 - acc: 0.4738 - val_loss: 1.0525 - val_acc: 0.5252

Epoch 00001: val_acc improved from -inf to 0.52520, saving model to /content/drive/My Drive/classification_Dataset/hackerearth_tag.h5
Epoch 2/10
281/281 [==============================] - 29s 102ms/step - loss: 1.0309 - acc: 0.5730 - val_loss: 0.9461 - val_acc: 0.6321

Epoch 00002: val_acc improved from 0.52520 to 0.63206, saving model to /content/drive/My Drive/classification_Dataset/hackerearth_tag.h5
Epoch 3/10
281/281 [==============================] - 30s 106ms/step - loss: 0.9885 - acc: 0.5963 - val_loss: 0.9432 - val_acc: 0.6018

Epoch 00003: val_acc did not improve from 0.63206
Epoch 4/10
281/281 [==============================] - 27s 95ms/step - loss: 0.9163 - acc: 0.6252 - val_loss: 0.8896 - val_acc: 0.6310

Epoch 00004: val_acc did not improve from 0.63206
Epoch 5/10
281/281 [==============================] - 27s 97ms/step - loss: 0.9050 - a

In [0]:
from keras.models import load_model
model_path='/content/drive/My Drive/classification_Dataset/hackerearth_tag.h5'
model_final.load_weights(model_path)

In [0]:
def prediction(img_path=test_images, model_path=model_path):
    model_final.load_weights(model_path)
    output=[]
    for img in listdir(img_path):
        #print(img)
        im=cv2.imread(img_path+img)
        im=cv2.resize(im,(image_dims[0],image_dims[1]))
        output.append([img,dummy_cols[np.argmax(model_final.predict(np.array([img_to_array(im)])))]])
        #print(im.shape)
    return output

In [78]:
output=prediction()

image9683.jpg
image8489.jpg
image6578.jpg
image6555.jpg
image10350.jpg
image3740.jpg
image3550.jpg
image10469.jpg
image7892.jpg
image1654.jpg
image4291.jpg
image2857.jpg
image1984.jpg
image3714.jpg
image1306.jpg
image5165.jpg
image631.jpg
image5832.jpg
image1486.jpg
image5706.jpg
image10468.jpg
image4323.jpg
image9062.jpg
image7616.jpg
image511.jpg
image1328.jpg
image10780.jpg
image2497.jpg
image5016.jpg
image1519.jpg
image6929.jpg
image9998.jpg
image6067.jpg
image5019.jpg
image10481.jpg
image2134.jpg
image3353.jpg
image9770.jpg
image4392.jpg
image7301.jpg
image5476.jpg
image4019.jpg
image7399.jpg
image10804.jpg
image3085.jpg
image6493.jpg
image7172.jpg
image6618.jpg
image2528.jpg
image5533.jpg
image8910.jpg
image10889.jpg
image7148.jpg
image6423.jpg
image1985.jpg
image5213.jpg
image336.jpg
image3112.jpg
image10752.jpg
image4939.jpg
image7767.jpg
image5673.jpg
image5488.jpg
image6040.jpg
image4583.jpg
image3221.jpg
image4924.jpg
image5502.jpg
image8827.jpg
image9743.jpg
image6684.jpg
i

In [0]:
pd.DataFrame(output, columns=['Image','Class']).to_csv('/content/drive/My Drive/classification_Dataset/sub1.csv', index=False)